In [1]:
!nvidia-smi

Fri Sep 10 08:20:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:0E:00.0 Off |                    0 |
| N/A   32C    P0    33W / 250W |  15525MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
import csv
import numpy as np
from PIL import Image
import batch_preparation
import keras 

In [4]:
data_folder = '/media/QNAP/People/GongyuZhang/ROP/data/raw/image/'

with open('../src/data/data_split.csv', 'r') as f:
    split = list(csv.reader(f))
    header = split[0]
    data = split[1:]

In [67]:
header

['file', 'subset', 'CH', 'JT', 'KP', 'SB', 'GA', 'group']

In [28]:
grading = {row[0]: row[-1] for row in data}

In [78]:
all_files = [row[0] for row in data]

In [81]:
all_data = {
    path: np.array(Image.open(os.path.join(data_folder, path)))[:,:,:3]
    for path in all_files
}

In [33]:
test_files = [row[0] for row in data if row[1] == 'test']

test_data_preprocessed = [(grading[path], batch_preparation.sample_patch(test_data[path], augment=False))
 for path in test_files]


In [48]:
best_models = {fold: max((m.split('_')[5]) for m in os.listdir('models') if m.split('_')[3] == str(fold))
               for fold in range(5)}
models = {
    fold: keras.models.load_model('models/val_fold_fold_{}_epoch_{}'.format(fold, best_models[fold]))
    for fold in range(5)
}

In [49]:
scores = np.array([
    [models[fold].predict_on_batch(x[np.newaxis])[0] for fold in range(5)]
    for (grading, x) in test_data_preprocessed])

In [65]:
results = []
for path, models_score in zip(test_files, scores):
    for fold in range(5):
        results.append((path, fold, *models_score[fold]))
    results.append((path, 'ensemble', *models_score.mean(axis=0)))

In [71]:
header_out = 'file', 'model', 'normal', 'pre-plus', 'plus'

In [75]:
with open('test_set_model_classification.csv', 'w') as f:
    csv.writer(f).writerows([header_out] + results)

In [82]:
all_data_preprocessed = [(grading[path], batch_preparation.sample_patch(all_data[path], augment=False))
 for path in all_files]

In [83]:
all_scores = [[models[fold].predict_on_batch(x[np.newaxis])[0] for fold in range(5)]
    for (grading, x) in all_data_preprocessed]

In [84]:
all_results = []
for path, models_score in zip(all_files, all_scores):
    for fold in range(5):
        all_results.append((path, fold, *models_score[fold]))


In [86]:
with open('model_classification_all_files.csv', 'w') as f:
    csv.writer(f).writerows([header_out] + all_results)